In [4]:
import numpy as np
import torch
import torch.nn as nn

In [5]:
x = torch.tensor(10.0)
x*2

tensor(20.)

Need to write differentiable approximations for the following operations: logical_and, indicate, max

# Soft Indicator Function

In [21]:
def compare(a, b):
    '''returns 1 if a > b and 2 if b > a'''
    def compare_soft(epsilon_t, nu_t, hardness=0.85):
        return torch.sigmoid(hardness * (epsilon_t - nu_t))

    def compare_hard(a, b):
        return (a > b).float()
    
    soft = compare_soft(a, b)
    return compare_hard(a, b) + soft - soft.detach()

def compare_hard(epsilon_t, nu_t):
    return (epsilon_t > nu_t).float()

a = torch.tensor(3.0, requires_grad=True)
b = torch.tensor(5.0, requires_grad=True)

x = compare(a, b)
x.backward()
print(a.grad, b.grad)

tensor(0.1110) tensor(-0.1110)


In [24]:
# def soft_maximum(value1, value2, hardness=0.85):
#     return torch.log(torch.exp(hardness * value1) + torch.exp(hardness * value2)) / hardness

def soft_maximum(a, b):
    return a*compare(a, b) + b*compare(b, a)

value1 = torch.tensor([3.0], requires_grad=True)
value2 = torch.tensor([5.0], requires_grad=True)

hard_max = torch.max(value1, value2)
soft_max = soft_maximum(value1, value2)

print(hard_max, soft_max)

soft_max.backward()
print(value1.grad, value2.grad)

tensor([5.], grad_fn=<MaximumBackward0>) tensor([5.], grad_fn=<AddBackward0>)
tensor([-0.2220]) tensor([1.2220])
